# Train Networks

- Three-way SoftMax or Multi-BCE classifier of normal, non-vascular MCI, and non-vascular dementia
- `Weights and Biases` sweep is used for hyperparameter search

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# Load some packages
import os
import json
from copy import deepcopy

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pprint
import wandb

# custom package
from models import *
from utils.cau_eeg_dataset import *
from utils.train_utils import *

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.10.1+cu113
cuda is available.


-----

## Set the default configuration for building datatset

In [4]:
cfg_data = {}
cfg_data['device'] = device
cfg_data['dataset'] = 'CAUHS'
cfg_data['data_path'] = r'dataset/02_Curated_Data_New/'
cfg_data['meta_path'] = os.path.join(cfg_data['data_path'], 'metadata_debug.json')
cfg_data['target_task'] = 'Normal, MCI, Dementia' # 'Norml, MCI, Dementia'
cfg_data['vascular'] = 'X'
cfg_data['segment'] = 'no' # 'train', 'all', 'no'
cfg_data['seed'] = 0
cfg_data['crop_length'] = 200 * 10 # 10 seconds
cfg_data['longer_crop_length'] = 200 * 10 * 10 # 100 seconds
cfg_data['crop_multiple'] = 4
cfg_data['minibatch'] = 64
cfg_data['input_norm'] = 'dataset' # 'datatset', 'datapoint', 'no'
cfg_data['EKG'] = 'O'
cfg_data['photic'] = 'X'
cfg_data['awgn'] = 5e-2
cfg_data['mgn'] = 1e-2
cfg_data['awgn_age'] = 5e-2

In [5]:
_ = build_dataset(cfg_data, verbose=True)

class_label_to_type: ['Normal', 'Non-vascular MCI', 'Non-vascular dementia']

----------------------------------------------------------------------------------------------------

- There are 458 data belonging to Normal
- There are 350 data belonging to Non-vascular MCI
- There are 235 data belonging to Non-vascular dementia

----------------------------------------------------------------------------------------------------

Train data label distribution	: [366, 280, 188] 834
Train data label distribution	: [46, 35, 24] 105
Train data label distribution	: [46, 35, 23] 104

----------------------------------------------------------------------------------------------------

composed_train: Compose(
)

----------------------------------------------------------------------------------------------------

composed_test: Compose(
)

----------------------------------------------------------------------------------------------------

longer_composed_test: Compose(
)

-----------------------

-----

## Define Network Models

In [6]:
if cfg_data.get('crop_multiple', 1) == 1:
    cfg_common_model = {'in_channels': _[0].dataset[0]['signal'].shape[0], 
                        'out_dims': len(_[-1])}
else:
    cfg_common_model = {'in_channels': _[0].dataset[0]['signal'][0].shape[0], 
                        'out_dims': len(_[-1])}
    
cfg_model_pool = []

#### 1D Tiny CNN

In [7]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-Tiny-CNN'
# cfg_model['generator'] = TinyCNN1D
# cfg_model['fc_stages'] = 1
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### M7 model (fc-age)

In [8]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-Mx'
# cfg_model['generator'] = M7
# cfg_model['fc_stages'] = 1
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 256
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### 1D ResNet model (fc-age)

In [9]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-ResNet-2x'
# cfg_model['generator'] = ResNet1D
# cfg_model['block'] = BottleneckBlock1D
# cfg_model['conv_layers'] = [2, 2, 2, 2]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### Deeper 1D ResNet model (fc-age)

In [10]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-ResNet-5x'
# cfg_model['generator'] = ResNet1D
# cfg_model['block'] = BottleneckBlock1D
# cfg_model['conv_layers'] = [3, 4, 6, 3]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### Shallower 1D ResNet

In [11]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-ResNet-2x'
# cfg_model['generator'] = ResNet1D
# cfg_model['block'] = BasicBlock1D
# cfg_model['conv_layers'] = [2, 2, 2, 2]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### Tiny 1D ResNet model

In [12]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-ResNet-1x'
# cfg_model['generator'] = ResNet1D
# cfg_model['block'] = BasicBlock1D
# cfg_model['conv_layers'] = [1, 1, 1, 1]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### Multi-Dilated 1D ResNet model

In [13]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-Multi-Dilated-ResNet-5x'
# cfg_model['generator'] = ResNet1D
# cfg_model['block'] = MultiBottleneckBlock1D
# cfg_model['conv_layers'] = [3, 4, 6, 3]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 32
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### 1D ResNeXt-53

In [14]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-5x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['LR'] = 1e-3
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': 0.001,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '1D-ResNeXt-5x',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), groups=32, bias=False

#### 1D ResNeXt-103

In [15]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-10x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['LR'] = 1e-3
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': 0.001,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '1D-ResNeXt-10x',
 'out_dims': 3,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (input_stage): Sequential(
    (0): Conv1d(20, 64, kernel_size=(27,), stride=(2,), padding=(13,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), groups=32, bias=Fal

#### 2D ResNet-20 model

In [16]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '2D-ResNet-2x' # resnet-18 + three more fc layer
# cfg_model['generator'] = ResNet2D
# cfg_model['block'] = BasicBlock2D
# cfg_model['conv_layers'] = [2, 2, 2, 2]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['n_fft'] = 100
# cfg_model['complex_mode'] = 'as_real' # 'power', 'remove'
# cfg_model['hop_length'] = cfg_model['n_fft'] // 2
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')

# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### 2D ResNet-52 model

In [17]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '2D-ResNet-5x' # resnet-50 + three more fc layer
# cfg_model['generator'] = ResNet2D
# cfg_model['block'] = Bottleneck2D
# cfg_model['conv_layers'] = [3, 4, 6, 3]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['n_fft'] = 100
# cfg_model['complex_mode'] = 'as_real' # 'power', 'remove'
# cfg_model['hop_length'] = cfg_model['n_fft'] // 2
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')

# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### 2D ResNeXt-104 model

In [18]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '2D-ResNeXt-10x' # resnet-101 + three more fc layer
# cfg_model['generator'] = ResNet2D
# cfg_model['block'] = Bottleneck2D
# cfg_model['conv_layers'] = [3, 4, 23, 3]
# cfg_model['fc_stages'] = 3
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['n_fft'] = 100
# cfg_model['complex_mode'] = 'as_real' # 'power', 'remove'
# cfg_model['hop_length'] = cfg_model['n_fft'] // 2
# cfg_model['groups'] = 32
# cfg_model['width_per_group'] = 8
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')

# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### CNN-Transformer

In [19]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-CNN-Transformer'
# cfg_model['generator'] = CNNTransformer
# cfg_model['fc_stages'] = 2
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 256
# cfg_model['n_encoders'] = 4
# cfg_model['n_heads'] = 4
# cfg_model['dropout'] = 0.2
# cfg_model['LR'] = 1e-3
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')

# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

#### Summarize the loaded models

In [20]:
for cfg_model in cfg_model_pool:
    pprint.pp(cfg_model, width=150)
    print('\n' + '-' * 100 + '\n')

{'in_channels': 20,
 'out_dims': 3,
 'model': '1D-ResNeXt-5x',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 64,
 'groups': 32,
 'LR': 0.001,
 'activation': 'relu'}

----------------------------------------------------------------------------------------------------

{'in_channels': 20,
 'out_dims': 3,
 'model': '1D-ResNeXt-10x',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 64,
 'groups': 32,
 'LR': 0.001,
 'activation': 'relu'}

----------------------------------------------------------------------------------------------------



-----

## Default Configurations for Training

In [21]:
# training configurations
cfg_train = {}
cfg_train['iterations'] = 100000
cfg_train['history_interval'] = cfg_train['iterations'] // 500
cfg_train['lr_decay_step'] = round(cfg_train['iterations'] * 0.8)
cfg_train['lr_decay_gamma'] = 0.1
cfg_train['weight_decay'] = 1e-2
cfg_train['mixup'] = 0.0 # 0 for no usage
cfg_train['criterion'] = 'cross-entropy' # 'cross-entropy', 'multi-bce'

cfg_train['device'] = device
cfg_train['save_model'] = True
cfg_train['save_temporary'] = False
cfg_train['draw_result'] = True
cfg_train['watch_model'] = False

In [22]:
def train_with_wandb(config, train_loader, val_loader, test_loader, test_loader_longer, class_label_to_type):
    print('*'*120)
    print(f'{"*"*30}{config["model"] + " train starts":^60}{"*"*30}')
    print('*'*120)

    # search an appropriate starting learning rate if needed
    model_state = None
    if config["LR"] is None:
        config['LR'], lr_search, model_state = learning_rate_search(config, train_loader, 
                                                                    min_log_lr=-4.5, max_log_lr=-3.0, 
                                                                    trials=100, steps=100)
        draw_learning_rate_record(lr_search, use_wandb=True)
    
    
    # generate model and its trainer
    model = config['generator'](**config).to(device)
    if model_state is not None:
        model.load_state_dict(model_state)
    
    optimizer = optim.AdamW(model.parameters(), 
                            lr=config['LR'], 
                            weight_decay=config['weight_decay'])
    scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                          step_size=config['lr_decay_step'], 
                                          gamma=config['lr_decay_gamma'])
    
    tr_ms = train_multistep if config.get('mixup', 0) < 1e-12 else train_mixup_multistep
    
    # track gradients and weights statistics
    if config.get('watch_model', None):
        wandb.watch(model, log='all', 
                    log_freq=config['history_interval'], 
                    log_graph=True)

    # train and validation routine
    best_val_acc = 0
    for i in range(0, config["iterations"], config["history_interval"]):
        # train 'history_interval' steps
        loss, train_acc = tr_ms(model, train_loader, optimizer, scheduler, 
                                config, config["history_interval"])

        # validation
        val_acc, _, _, _, _ = check_accuracy(model, val_loader, config, repeat=10)

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_model_state = deepcopy(model.state_dict())
            if config['save_model'] and config['save_temporary']:
                save_path = f'checkpoint_temp/{wandb.run.name}/'
                os.makedirs(save_path, exist_ok=True)
                path = os.path.join(save_path, f'{config["model"]}')
                torch.save(best_model_state, path)

        # log
        wandb.log({'Loss': loss, 
                   'Train Accuracy': train_acc, 
                   'Validation Accuracy': val_acc}, step=i + config["history_interval"])

    # calculate the test accuracies for best and last models
    last_model_state = deepcopy(model.state_dict())
    last_test_result = check_accuracy(model, test_loader, config, repeat=30)
    last_test_acc = last_test_result[0]

    model.load_state_dict(best_model_state)
    best_test_result = check_accuracy(model, test_loader, config, repeat=30)
    best_test_acc = best_test_result[0]

    if last_test_acc < best_test_acc:
        model_state = best_model_state
        test_result = best_test_result
    else:
        model_state = last_model_state
        test_result = last_test_result

    model.load_state_dict(model_state)
    test_acc, test_confusion, test_debug, score, target = test_result

    # calculate the test accuracies for final model on much longer sequence
    longer_test_acc = check_accuracy(model, test_loader_longer, config, repeat=30)[0]

    # save the model
    if config['save_model']:
        save_path = f'checkpoint_temp/{wandb.run.name}/'
        os.makedirs(save_path, exist_ok=True)
        path = os.path.join(save_path, f'{config["model"]}')
        torch.save(model_state, path)

    # leave the message
    wandb.config.final_shape = model.get_final_shape()
    wandb.config.num_params = count_parameters(model)
    wandb.log({'Test Accuracy': test_acc,
               '(Best / Last) Test Accuracy': ('Best' if last_test_acc < best_test_acc else 'Last', 
                                               round(best_test_acc, 2), round(last_test_acc, 2)),
               'Confusion Matrix (Array)': test_confusion,
               'Test Accuracy (Longer)': longer_test_acc, 
               'Test Debug Table/Serial': test_debug[0], 
               'Test Debug Table/EDF': test_debug[1], 
               'Test Debug Table/Pred': test_debug[2], 
               'Test Debug Table/GT': test_debug[3]})

    if config['draw_result']:
        draw_roc_curve(score, target, class_label_to_type, use_wandb=True)
        draw_confusion(test_confusion, class_label_to_type, use_wandb=True)
        draw_debug_table(test_debug, use_wandb=True)
        # leave these disabled to save the wandb resources
        # wandb.log({"Confusion Matrix": wandb.plot.confusion_matrix(y_true=target, 
        #                                                            preds=score.argmax(axis=-1), 
        #                                                            class_names=class_label_to_type)})
        # wandb.log({"ROC Curve": wandb.plot.roc_curve(target, score, labels=class_label_to_type)})
        
    return model

In [23]:
def train_sweep(cfg_data, cfg_train, cfg_model_pool):
    wandb_run = wandb.init()
    wandb.run.name = wandb.run.id
    with wandb_run:
        # wandb config update
        cfg_model = cfg_model_pool[wandb.config.model_index]
        config = {}
        for k, v in {**cfg_data, **cfg_train, **cfg_model}.items():
            if k not in wandb.config:
                config[k] = v
        
        # to prevent callables from type-conversion to str
        wandb.config.update(config)
        for k, v in wandb.config.items():
            if k not in config:
                config[k] = v
        config['history_interval'] = config['iterations'] // 500
                
        # build dataset
        train_loader, val_loader, test_loader, test_loader_longer, class_label_to_type = build_dataset(config)        
        
        if cfg_data.get('crop_multiple', 1) == 1:
            config['in_channels'] = train_loader.dataset[0]['signal'].shape[0]
        else:
            config['in_channels'] = train_loader.dataset[0]['signal'][0].shape[0]
        config['out_dims'] = len(class_label_to_type)
        
        # train the model
        train_with_wandb(config, train_loader, 
                         val_loader, test_loader, 
                         test_loader_longer, class_label_to_type)

-----

## Train

In [24]:
sweep_data = {}
sweep_data['crop_length'] = {
    'values': [
        # 200 * 10, # 10 sec
        200 * 15, # 15 sec
        200 * 20, # 20 sec
        # 200 * 30, # 30 sec
              ],
}

sweep_data['EKG'] = {
    'values': ['O', 'X'],
}

sweep_data['photic'] = {
    'values': ['O', 'X'],
}

sweep_data['awgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

sweep_data['mgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.1,
}

sweep_data['awgn_age'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

In [25]:
sweep_model = {}
sweep_model['model_index'] = { 
    'values' : [i for i in range(len(cfg_model_pool))] 
}

sweep_model['fc_stages'] = { 
    'distribution' : 'int_uniform',
    'min': 1,
    'max': 4,
}

sweep_model['use_age'] = { 
    'values' : ['fc', 'conv']
}

sweep_model['final_pool'] = { 
    'values' : ['max', 'average']
}

sweep_model['first_dilation'] = { 
    'distribution' : 'int_uniform',
    'min': 1,
    'max': 3,
}

sweep_model['base_stride'] = { 
    'distribution' : 'int_uniform',
    'min': 3,
    'max': 6,
}

sweep_model['base_channels'] = { 
    'values' : [32, 64]
}

sweep_model['dropout'] = {
    'distribution': 'uniform',
    'min': 0.3,
    'max': 0.5
}

# sweep_model['LR'] = {
#     'distribution': 'log_uniform',
#     'min': math.log(5e-5),
#     'max': math.log(2e-3)
# }

sweep_model['activation'] = {
    'values' : ['relu', 'gelu', 'mish']
}

In [26]:
sweep_train = {}
sweep_train['iterations'] = {
    'values' : [150000]
}

sweep_train['lr_decay_gamma'] = {
    'distribution' : 'uniform',
    'min': 0.1,
    'max': 0.3,
}

sweep_train['lr_decay_step'] = {
    'values' : [120000]
}

sweep_train['weight_decay'] = {
    'distribution' : 'log_uniform',
    'min': math.log(1e-5),
    'max': math.log(1e-1)
}

sweep_train['mixup'] = {
    'values': [0, 0.1, 0.2, 0.3]
}

sweep_train['criterion'] = {
    'values': ['cross-entropy', 'multi-bce']
}

In [27]:
sweep_config = {
    "entity": "ipis-mjkim",
    "name" : "my-sweep",
    "method" : "random",
    "parameters" : 
    {
        **sweep_data,
        **sweep_model,
        **sweep_train,
    }
}

sweep_id = wandb.sweep(sweep_config, project="eeg-analysis-2")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 5xedutd0
Sweep URL: https://wandb.ai/ipis-mjkim/eeg-analysis-2/sweeps/5xedutd0


In [ ]:
wandb.agent(sweep_id, function=lambda: train_sweep(cfg_data, cfg_train, cfg_model_pool), count=100)

wandb: Agent Starting Run: 6aupyg89 with config:
wandb: 	EKG: X
wandb: 	activation: gelu
wandb: 	awgn: 0.022450589913571083
wandb: 	awgn_age: 0.06883500831347242
wandb: 	base_channels: 32
wandb: 	base_stride: 5
wandb: 	criterion: cross-entropy
wandb: 	crop_length: 4000
wandb: 	dropout: 0.3282229003710398
wandb: 	fc_stages: 3
wandb: 	final_pool: max
wandb: 	first_dilation: 1
wandb: 	iterations: 150000
wandb: 	lr_decay_gamma: 0.2515588794670761
wandb: 	lr_decay_step: 120000
wandb: 	mgn: 0.04544937556456111
wandb: 	mixup: 0.2
wandb: 	model_index: 0
wandb: 	photic: X
wandb: 	use_age: conv
wandb: 	weight_decay: 7.367592236027656e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ipis-mjkim (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


************************************************************************************************************************
******************************                 1D-ResNeXt-5x train starts                 ******************************
************************************************************************************************************************
